In [7]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from mpl_toolkits import mplot3d
from matplotlib import rcParams
import copy
import math

In [16]:
class surface:
  def __init__( self, equation ):
    self.equation = equation

  def get_equation():
    return self.equation;

  def get_equation_eval( x, y ):
    return self.equation( x, y )

In [22]:
class lens_ellipse_circular( surface ):
  def __init__( self, a, b ):
    super().__init__( self.equation_3d )
    self.a = a
    self.b = b

  def equation_2d( r ):
    return ( self.b**2 - ( r*self.b/self.a )**2 )**(1/2)

  def equation_3d( x, y ):
    r = ( x**2 + y**2 )**(1/2)
    return equation_ellipse_2d( r )

  def get_position_incidence_2d( photon_yIntercept, photon_slope ):
    x = ( -photon_slope * photon_yIntercept + self.b * ( photon_slope**2 + self.b**2/self.a**2 - photon_yIntercept**2/self.a**2 )**(1/2) ) / ( m**2 + self.b**2/self.a**2 )
    y = equation_2d( x )
    return ( x, y )

  def get_position_incidence_3d( photon_position, photon_direction ):
    # rz-plane where r2=x2+y2
    r_photon_direction = ( photon_direction[ 0 ]**2 + photon_direction[ 1 ]**2 )**(1/2)
    z_photon_direction = photon_direction[ 2 ]
    photon_slope = z_photon_direction / r_photon_direction

    r_photon_position = ( photon_position[ 0 ]**2 + photon_position[ 1 ]**2 )**(1/2)
    z_photon_position = photon_position[ 0 ]
    photon_yIntercept = z_photon_position - photon_slope * r_photon_position

    ( r_incidence, z ) = self.get_position_incidence_2d( photon_yIntercept, photon_slope )

    # xy-plane
    photon_slope = photon_direction[ 1 ] / photon_direction[ 0 ] # redefinition of xy-slope
    photon_yIntercept = photon_position[ 1 ] - photon_slope * photon_position[ 1 ]
    x = ( -photon_slope * photon_yIntercept + ( -m**2 * r_incidence**2 - photon_yIntercept**2 + r_incidence**2 )**(1/2) ) / ( photon_slope**2 + 1 )
    y = ( r_incidence**2 - x**2 )**(1/2)

    return ( x, y, z )

In [25]:
def get_reflection_s( n_1, n_2, theta_i ):
  cos_theta_i = math.cos( theta_i )
  sin_theta_i = math.sin( theta_i )
  numerator   = n_1 * cos_theta_i - n_2 * math.sqrt( 1 - ( n_1 / n_2 * sin_theta_i ) ** 2 )
  denominator = n_1 * cos_theta_i + n_2 * math.sqrt( 1 - ( n_1 / n_2 * sin_theta_i ) ** 2 )
  return ( numerator / denominator ) ** 2

def get_reflection_p( n_1, n_2, theta_i ):
  cos_theta_i = math.cos( theta_i )
  sin_theta_i = math.sin( theta_i )
  numerator   = n_1 * ( 1 - ( n_1 / n_2 * sin_theta_i )**2 )**(1/2) - n_2 * cos_theta_i
  denominator = n_1 * ( 1 - ( n_1 / n_2 * sin_theta_i )**2 )**(1/2) + n_2 * cos_theta_i
  return ( numerator / denominator ) ** 2

def get_reflection( n_1, n_2, theta_i, spRatio=1 ): # spRatio = 1 for natural light
  return spRatio * get_reflection_s( n_1, n_2, theta_i ) / get_reflection_p( n_1, n_2, theta_i )

class lens:
  def __init__( self, position, direction, surface_1, surface_2 ):
    self.position = position
    self.direction = direction
    self.surface_1 = surface_1
    self.surface_2 = surface_2
  
  def get_position():
    return self.position

  def get_direction():
    return self.direction

  def get_surface_1():
    return self.surface_1;
  
  def get_surface_2():
    return self.surface_2;

  def get_position_incidence_surface_1( photon_position, photo_direction ):
    return surface_1.get_position_incidence_3d( photon_position, photo_direction )

  def get_angle_reflection_surface_1( indexRefraction_1, indexRefraction_2, angle_incidence ):
    return get_reflection( indexRefraction_1, indexRefraction_2, angle_incidence, 1 )
  
  def get_angle_reflection_surface_1( photon_position, photon_direction ):
    return "here"

  def get_angle_incidence_surface_1( photon_position, photo_direction ):
    return "here"
  
  def get_angle_refraction( angle_incidence ):
    return "here"